* Importando os pecotess necessários

In [60]:
import urllib, urllib.error
from functions import save_in_database
from urllib import parse
import json
import requests
import sqlite3
import pandas as pd

* Define uma função que recebe um dicionário JSON e imprime informações específicas desse dicionário, desde que essas informações correspondam às chaves especificadas na lista list_keys. Ele cria uma apresentação organizada das informações do filme, separadas por hifens, para facilitar a leitura e compreensão.

In [61]:
def print_json(json_data):
    list_keys=['Title', 'Year', 'Rated', 'Released', 'Runtime', 'Genre', 'Director', 'Writer', 
               'Actors', 'Plot', 'Language', 'Country', 'Awards',
               'Metascore', 'imdbRating', 'imdbVotes', 'imdbID']
    print("-"*50)
    for i in list_keys:
        if i in list(json_data.keys()):
            print(f"{i}: {json_data[i]}")
    print("-"*50)

In [62]:
def search_movie(title):
    if len(title) < 1 or title=='quit': 
        print("Não encontrado...")
        return 

    try:
        title_url = parse.urlencode({'t': title})
        api_key = '9019e34'
        url = f'http://www.omdbapi.com/?{title_url}&apikey={api_key}'
        print(f'Procurando o filme "{title}"... ')
        uh = urllib.request.urlopen(url)
        data = uh.read()
        json_data=json.loads(data)
        
        if json_data['Response']=='True':
            print_json(json_data)
            save_database =input('Salvar o filme no banco de dados? Digite "sim" ou "não": ').lower()
            if save_database =='sim':
                save_in_database(json_data)
        else:
            print("Erro encontrado: ",json_data['Erro'])
    
    except urllib.error.URLError as e:
        print(f"ERROR: {e.reason}")

In [63]:
def save_in_database(movie_data):
    conn = sqlite3.connect('info_movie.sqlite')
    cur=conn.cursor()

    cur.execute ('''CREATE TABLE IF NOT EXISTS filmes (
        title TEXT,
        year INTEGER,
        rated TEXT,
        released TEXT,
        runtime TEXT,
        genre TEXT,
        director TEXT,
        writer TEXT,
        actors TEXT,
        plot TEXT,
        language TEXT,
        country TEXT,
        awards TEXT,
        metascore INTEGER,
        imdbRating REAL,
        imdbVotes TEXT,
        imdbID TEXT
    )
''')
    conn.execute('INSERT INTO filmes VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)',
             (movie_data['Title'], movie_data['Year'], movie_data['Rated'], movie_data['Released'], movie_data['Runtime'],
              movie_data['Genre'], movie_data['Director'], movie_data['Writer'], movie_data['Actors'], movie_data['Plot'],
              movie_data['Language'], movie_data['Country'], movie_data['Awards'], movie_data['Metascore'],
              movie_data['imdbRating'], movie_data['imdbVotes'], movie_data['imdbID']))
    conn.commit()
    conn.close()

* Pegar a chave da API

In [64]:
title = input('\nDigite o nome do filme: ')
if len(title) < 1 or title=='quit': 
    print("Não encontrado...")
else:
    movie_data = search_movie(title)
    if movie_data is not None:
        save_in_database(movie_data)

Procurando o filme "Elementos"... 
--------------------------------------------------
Title: Reto 4 elementos: naturaleza extrema
Year: 2018–
Rated: N/A
Released: 19 Mar 2018
Runtime: N/A
Genre: Action, Adventure, Family
Director: N/A
Writer: Grecia Feliciano Murrieta
Actors: Montserrat Oliver, Adrian Di Monte, Cristián de la Fuente
Plot: The show is mainly about a couple color groups competing in a variety of obstacles trying to not be eliminated which will take them out of the competition or punish their standings. The best group is set out to win and win a certa...
Language: Spanish
Country: Mexico
Awards: 1 nomination
Metascore: N/A
imdbRating: 8.1
imdbVotes: 35
imdbID: tt8506878
--------------------------------------------------
